# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

The model takes a text as input and performs some kind of analysis. So this could be extracting labels, extracting names, kind of understanding the sentiment of a text.

So if you want to extract a sentiment, positive or negative, of a piece of text, in the traditional 
machine learning workflow, you'd have to collect the label data set, train 
a model, figure out how to deploy the model somewhere in 
the cloud and make inferences. And that could work pretty well, but 
it was, you know, just a lot of work to 
go through that process. And also for every task, such 
as sentiment versus extracting names versus 
something else, you have to train and deploy a separate model.

One of the really nice 
things about large language model is that for 
many tasks like these, you can just write a 
prompt and have it start generating results pretty 
much right away. And that gives tremendous speed in terms 
of application development. And you can also just use one model, one 
API to do many different tasks rather than 
needing to figure out how to train and deploy a lot of 
different models.

## Setup

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*

*The sentiment of the product review is positive.*

It's pretty right.

In [ ]:
# If you want more concise response to make it easier for post-processing

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*

*positive*

## Identify types of emotions

Large language models are pretty good at extracting 
specific things out of a piece of text.

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*

*satisfied, pleased, grateful, impressed, happy*

## Identify anger

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*

*No*

Notice that 
with supervised learning, if I had 
wanted to build all of these classifiers, there's no way 
I would have been able to do this with 
supervised learning in the just a few minutes.

## Extract product and company name from customer reviews

Information extraction is the part of NLP, 
of Natural Language Processing, that relates to taking 
a piece of text and extracting certain things 
that you want to know from the text.

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*
```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```
you can easily load this 
into the Python dictionary to then do additional processing 
on this output.

## Doing multiple tasks at once

Write a prompt:
- to recognize the sentiment, 
- figure out if someone is angry,
- then also extract the item and the brand.
Write a 
single prompt to extract all of this information 
at the same time

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*
```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```

## Inferring topics

In [ ]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*

1. Government survey
2. Department satisfaction rating
3. NASA
4. Social Security Administration
5. Job satisfaction improvement

In [ ]:
response.split(sep=',')

*OUTPUT*
```python
['1. Government survey\n2. Department satisfaction rating\n3. NASA\n4. Social Security Administration\n5. Job satisfaction improvement']
```

You can also split it so you get a Python list with the five topics that 
this article was about. 
And if you have a collection of articles and extract 
topics, you can then also use a large language 
model to help you index into different topics.

In [ ]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as json with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

*OUTPUT*
```json
{
  "nasa": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 1,
  "federal government": 1
}
```

In machine learning, this is sometimes called a "Zero-Shot Learning Algorithm", 
because we didn't give it any training data that was 
labeled, so that's Zero-Shot. And with just a prompt, it 
was able to determine which of these topics are covered in that news article.

topic_dict =


nasa : 1
local goverment : 0
engineering : 0
employee satisfaction : 1
federal goverment : 1

In [ ]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

*OUTPUT*

*ALERT: New NASA story!*

And in just a few minutes, you 
can build multiple systems for making inferences about text 
that previously just would have taken days or even 
weeks for a skilled machine learning developer.